## data load, sentence split

In [17]:
from utils.data_preprocess_utils import load_jsonl,split_sentences

In [ ]:
load_data = load_jsonl('../data/preprocess_test_data.jsonl')
print(f"original_data, size = {len(load_data)}")

In [ ]:
doc_sentences = split_sentences(load_data)
print("splitted sentence: \n")
for i in range(0, len(load_data)):
     print(f"multi-doc samples {i + 1} has {len(doc_sentences[i])} docs.")
     for doc in doc_sentences[i]:
          print(f"sentence number: {len(doc)}")
          print(f"document sentences: {doc}")

     print("\n")

## keyword extract

In [ ]:
from utils.data_preprocess_utils import extract_keywords



### method try

In [ ]:
from keybert import KeyBERT

doc = """
          Supervised learning is the machine learning task of learning a function that
          maps an input to an output based on example input-output pairs.[1] It infers a
          function from labeled training data consisting of a set of training examples.[2]
          In supervised learning, each example is a pair consisting of an input object
          (typically a vector) and a desired output value (also called the supervisory signal).
          A supervised learning algorithm analyzes the training data and produces an inferred function,
          which can be used for mapping new examples. An optimal scenario will allow for the
          algorithm to correctly determine the class labels for unseen instances. This requires
          the learning algorithm to generalize from the training data to unseen situations in a
          'reasonable' way (see inductive bias).
          """
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

In [ ]:
print(keywords)

In [ ]:
# sentence keyword
s1 = "Supervised learning is the machine learning task of learning a function that maps an input to an output based on example input-output pairs."
s2 = "I am a women."
s3 = "hei!"
kw1 = kw_model.extract_keywords(s1)
kw2 = kw_model.extract_keywords(s2)
kw3 = kw_model.extract_keywords(s3)

print(f"sentence keywords: \n kw1: {kw1} \n kw2: {kw2} \n kw3: {kw3}")


In [ ]:
## stop word remove
import spacy

nlp = spacy.load("en_core_web_sm")
# 获取 spaCy 的停用词列表
stop_words = nlp.Defaults.stop_words

kw1_noStopWord = [(word, score) for word, score in kw1 if word.lower() not in stop_words]
kw2_noStopWord = [(word, score) for word, score in kw2 if word.lower() not in stop_words]
kw3_noStopWord = [(word, score) for word, score in kw3 if word.lower() not in stop_words]
print(f"sentence keywords without stopwords: \n kw1: {kw1_noStopWord} \n kw2: {kw2_noStopWord} \n kw3: {kw3_noStopWord}")

In [ ]:
print(len(stop_words))

In [ ]:
def calculate_keyword_count(text, words_per_100=1, min_keywords=2, max_keywords=1100):
     # 计算文本长度
     text_length = len(text)
     print(text_length)
     # 按比例计算关键词数量
     keyword_count = max(min_keywords, min(max_keywords, text_length // 4 * words_per_100))
     
     return keyword_count

# 示例
text = "这是一段示例文本，用于演示如何根据文本长度动态调整关键词数量。"
keyword_count = calculate_keyword_count(text)
print(f"建议提取的关键词数量: {keyword_count}")

## coreference resolve

In [18]:
## new method try
import coreferee
import spacy

nlp_l = spacy.load('en_core_web_lg')
nlp_l.add_pipe('coreferee')
doc = nlp_l('Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much. She also meet Tom, Anny and John, who are their previous colleges')
# print(type(doc))
doc._.coref_chains.print()
# print(type(doc._.coref_chains[0]))

# for chain in doc._.coref_chains:
#      print(f"current chain: {chain}. \n its mentions are: ")
#      for mention in chain:
#           print(mention)
          
# print(doc._.coref_chains.resolve(doc[31]))

# print(doc._.coref_chains[0].pretty_representation)
ind = doc._.coref_chains[5][1][0]
print(f"index: {ind}, ancetedent: {doc._.coref_chains.resolve(doc[ind])}") #  ancetedent: None

0: he(1), his(6), Peter(9), He(16), his(18)
1: work(7), it(14)
2: [He(16); wife(19)], they(21), They(26), they(31)
3: wife(19), She(38)
4: Spain(29), country(34)
5: [Tom(41); Anny(43); John(45)], their(49)
index: 49, ancetedent: [Tom, Anny, John]


In [ ]:
chain = doc._.coref_chains[0]
print(f"chain 1 antecend: {chain.most_specific_mention_index}, token_id:{chain[chain.most_specific_mention_index]}")

In [19]:
print(f"{doc[34]} is in the sent: {doc[34].sent}")

country is in the sent: They travelled to Spain because they loved the country very much.


## over-all test

In [41]:
%load_ext autoreload
%autoreload 2

from utils.data_preprocess_utils import define_node_edge, extract_keywords, coref_resolve

2025-01-22 20:34:12,810 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu - SentenceTransformer.py:210
2025-01-22 20:34:12,810 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2 - SentenceTransformer.py:218


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
test_docs_list = [[
     ["Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much."],
     ["A stray cat appeared in the garden one rainy day. Sarah offered milk and a blanket. It stayed, becoming part of her family."],
     ["The bakery’s croissants were legendary. Every morning, the scent of butter drew a crowd. Mrs. Laurent, the baker, perfected each one. Customers claimed they were the best they’d ever tasted."]
]]

In [ ]:
#### doc1
'''
Although he was very busy with his work, Peter had had enough of it. n1,sen1
He and his wife decided they needed a holiday. n2,sen2
They travelled to Spain because they loved the country very much. n3,sen3
'''
####
#### doc2
'''
A stray cat appeared in the garden one rainy day. n4,sen1
Sarah offered milk and a blanket. n5,sen2
It stayed, becoming part of her family. n6,sen3
'''
####
#### doc3
'''
The bakery’s croissants were legendary. n7,sen1
Every morning, the scent of butter drew a crowd. n8,sen2
Mrs. Laurent, the baker, perfected each one. n9,sen3
Customers claimed they were the best they’d ever tasted. n10,sen4
'''
####

In [6]:
docs_kws = extract_keywords(test_docs_list)

for docs in docs_kws:
     for doc in docs:
          print("final result: ", doc)

2025-01-22 17:29:04,891 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu - SentenceTransformer.py:210
2025-01-22 17:29:04,891 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2 - SentenceTransformer.py:218


final result:  [('peter', 0.4611), ('spain', 0.2796)]
final result:  [('stray', 0.4608), ('milk', 0.1903)]
final result:  [('croissants', 0.6856), ('laurent', 0.284)]


In [15]:
corfs = coref_resolve(test_docs_list)

for docs in corfs:
     for doc in docs:
          print("doc final result: ")
          for cluster in doc:
               print(cluster)

2025-01-22 19:17:02,420 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu - SentenceTransformer.py:210
2025-01-22 19:17:02,423 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2 - SentenceTransformer.py:218


chain:  0: he(1), his(6), Peter(9), He(16), his(18)
ant: 9: 
 text: Although he was very busy with his work, Peter had had enough of it.
chain:  1: work(7), it(14)
ant: 7: 
 text: Although he was very busy with his work, Peter had had enough of it.
chain:  2: [He(16); wife(19)], they(21), They(26), they(31)
ant: 16: 
 text: He and his wife decided they needed a holiday.
chain:  3: Spain(29), country(34)
ant: 29: 
 text: They travelled to Spain because they loved the country very much.
chain:  0: cat(2), It(18)
ant: 2: 
 text: A stray cat appeared in the garden one rainy day.
chain:  1: Sarah(11), her(24)
ant: 11: 
 text: Sarah offered milk and a blanket.
chain:  0: Customers(28), they(30), they(34)
ant: 28: 
 text: Customers claimed they were the best they’d ever tasted.
doc final result: 
[(0, 0, Although he was very busy with his work, Peter had had enough of it.), (0, 0, He and his wife decided they needed a holiday.)]
[(0, 0, He and his wife decided they needed a holiday.), (0, 0, 

In [48]:
word_map, sent_map, edges = define_node_edge(test_docs_list)

2025-01-22 20:57:28,982 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu - SentenceTransformer.py:210
2025-01-22 20:57:29,011 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2 - SentenceTransformer.py:218


In [49]:
print("word map: ", word_map)
print("sent_map: ", sent_map)

word map:  [{'peter': 10, 'spain': 11, 'stray': 12, 'milk': 13, 'croissants': 14, 'laurent': 15}]
sent_map:  [{(0, 0, 'Although he was very busy with his work, Peter had had enough of it.'): 0, (0, 0, 'He and his wife decided they needed a holiday.'): 1, (0, 0, 'They travelled to Spain because they loved the country very much.'): 2, (0, 1, 'A stray cat appeared in the garden one rainy day.'): 3, (0, 1, 'Sarah offered milk and a blanket.'): 4, (0, 1, 'It stayed, becoming part of her family.'): 5, (0, 2, 'The bakery’s croissants were legendary.'): 6, (0, 2, 'Every morning, the scent of butter drew a crowd.'): 7, (0, 2, 'Mrs. Laurent, the baker, perfected each one.'): 8, (0, 2, 'Customers claimed they were the best they’d ever tasted.'): 9}]


In [54]:
for key, value in word_map[0].items():
     print(f"{key}: {value}")

peter: 10
spain: 11
stray: 12
milk: 13
croissants: 14
laurent: 15


In [59]:
for (k1,k2), value in edges[0].items():
     for e in value:
          print(f"{k1}, {k2}: {e}")

10, 0: {'type': 'word_sent', 'weight': 0.4611}
11, 2: {'type': 'word_sent', 'weight': 0.2796}
12, 3: {'type': 'word_sent', 'weight': 0.4608}
13, 4: {'type': 'word_sent', 'weight': 0.1903}
14, 6: {'type': 'word_sent', 'weight': 0.6856}
15, 8: {'type': 'word_sent', 'weight': 0.284}
1, 0: {'type': 'pronoun_antecedent', 'weight': 1}
2, 1: {'type': 'pronoun_antecedent', 'weight': 1}
5, 3: {'type': 'pronoun_antecedent', 'weight': 1}
5, 4: {'type': 'pronoun_antecedent', 'weight': 1}
